In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [ ]:
# Sample data for training
conversations = [
    ("Hi", "Hello! How can I help you today?"),
    ("What products do you have?", "We offer a variety of clothing and accessories."),
    ("Tell me about your dresses", "Our dresses are designed with the latest fashion trends in mind."),
    # Add more conversation pairs as needed
]

In [ ]:
# Preprocess the data
input_texts = [text[0] for text in conversations]
target_texts = [text[1] for text in conversations]

input_vocab = sorted(set(" ".join(input_texts)))
target_vocab = sorted(set(" ".join(target_texts)))

input_token_index = dict([(char, i) for i, char in enumerate(input_vocab)])
target_token_index = dict([(char, i) for i, char in enumerate(target_vocab)])

max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, len(input_vocab)), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, len(target_vocab)), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, len(target_vocab)), dtype="float32")

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

In [ ]:
# Build the model
model = Sequential()
model.add(LSTM(256, input_shape=(None, len(input_vocab))))
model.add(Dense(len(target_vocab), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train the model
model.fit(encoder_input_data, decoder_target_data, epochs=100, batch_size=16)

Epoch 1/100


ValueError: ignored

In [ ]:
# Function to generate responses
def generate_response(input_text):
    input_seq = np.zeros((1, max_encoder_seq_length, len(input_vocab)), dtype="float32")
    for t, char in enumerate(input_text):
        input_seq[0, t, input_token_index[char]] = 1.0

    decoded_sequence = ""
    for t in range(max_decoder_seq_length):
        output_probs = model.predict(input_seq)
        sampled_token_index = np.argmax(output_probs[0, t, :])
        sampled_char = target_vocab[sampled_token_index]
        decoded_sequence += sampled_char

        if sampled_char == "\n":
            break

        input_seq[0, t + 1, sampled_token_index] = 1.0

    return decoded_sequence


In [ ]:
# Test the chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    response = generate_response(user_input)
    print("Bot:", response)